In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('C:/Users/Debian_Boy/OneDrive/Desktop/BTC_data/BTC-2021min.csv', date_parser = True)
print(data)


              unix                 date   symbol      open      high  \
0       1646106180  2022-03-01 03:43:00  BTC/USD  43046.58  43046.58   
1       1646106060  2022-03-01 03:41:00  BTC/USD  43018.23  43046.59   
2       1646106000  2022-03-01 03:40:00  BTC/USD  43022.24  43022.24   
3       1646105940  2022-03-01 03:39:00  BTC/USD  43035.16  43035.16   
4       1646105880  2022-03-01 03:38:00  BTC/USD  43077.82  43077.82   
...            ...                  ...      ...       ...       ...   
610777  1609459500  2021-01-01 00:05:00  BTC/USD  29021.86  29023.38   
610778  1609459440  2021-01-01 00:04:00  BTC/USD  29048.13  29057.73   
610779  1609459380  2021-01-01 00:03:00  BTC/USD  29037.68  29069.39   
610780  1609459320  2021-01-01 00:02:00  BTC/USD  29069.80  29073.02   
610781  1609459260  2021-01-01 00:01:00  BTC/USD  29007.31  29086.90   

             low     close  Volume BTC     Volume USD  
0       43046.58  43046.58    0.000000       0.000000  
1       43018.23  43046

In [2]:
data.tail()
#print(data)
#data training
#data_training = data[data['date']< '2022-02-22 05:02:00'].copy()
#print(data_training)

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
610777,1609459500,2021-01-01 00:05:00,BTC/USD,29021.86,29023.38,28982.33,28999.50,1.062360,30807.899250
610778,1609459440,2021-01-01 00:04:00,BTC/USD,29048.13,29057.73,29035.61,29045.19,1.446538,42014.959434
610779,1609459380,2021-01-01 00:03:00,BTC/USD,29037.68,29069.39,29019.00,29048.13,2.189621,63604.382387
610780,1609459320,2021-01-01 00:02:00,BTC/USD,29069.80,29073.02,29028.14,29035.89,3.030301,87987.499279
610781,1609459260,2021-01-01 00:01:00,BTC/USD,29007.31,29086.90,29007.31,29083.47,14.561951,423512.060397


In [3]:
#print(data)
#data training
data_training = data[data['date']< '2022-02-22 05:02:00'].copy()
print(data_training)

              unix                 date   symbol      open      high  \
10001   1645506060  2022-02-22 05:01:00  BTC/USD  36676.28  36676.28   
10002   1645506000  2022-02-22 05:00:00  BTC/USD  36646.81  36686.82   
10003   1645505940  2022-02-22 04:59:00  BTC/USD  36628.40  36642.71   
10004   1645505880  2022-02-22 04:58:00  BTC/USD  36615.01  36649.31   
10005   1645505820  2022-02-22 04:57:00  BTC/USD  36653.02  36653.02   
...            ...                  ...      ...       ...       ...   
610777  1609459500  2021-01-01 00:05:00  BTC/USD  29021.86  29023.38   
610778  1609459440  2021-01-01 00:04:00  BTC/USD  29048.13  29057.73   
610779  1609459380  2021-01-01 00:03:00  BTC/USD  29037.68  29069.39   
610780  1609459320  2021-01-01 00:02:00  BTC/USD  29069.80  29073.02   
610781  1609459260  2021-01-01 00:01:00  BTC/USD  29007.31  29086.90   

             low     close  Volume BTC     Volume USD  
10001   36669.51  36671.91    0.185255    6793.654320  
10002   36646.81  36686

In [4]:
data_test = data[data['date']> '2022-02-22 05:02:00'].copy()
print(data_test)

            unix                 date   symbol      open      high       low  \
0     1646106180  2022-03-01 03:43:00  BTC/USD  43046.58  43046.58  43046.58   
1     1646106060  2022-03-01 03:41:00  BTC/USD  43018.23  43046.59  43018.23   
2     1646106000  2022-03-01 03:40:00  BTC/USD  43022.24  43022.24  43016.03   
3     1646105940  2022-03-01 03:39:00  BTC/USD  43035.16  43035.16  42999.44   
4     1646105880  2022-03-01 03:38:00  BTC/USD  43077.82  43077.82  43049.46   
...          ...                  ...      ...       ...       ...       ...   
9995  1645506420  2022-02-22 05:07:00  BTC/USD  36653.99  36706.29  36649.03   
9996  1645506360  2022-02-22 05:06:00  BTC/USD  36699.96  36700.00  36648.15   
9997  1645506300  2022-02-22 05:05:00  BTC/USD  36687.82  36702.32  36687.82   
9998  1645506240  2022-02-22 05:04:00  BTC/USD  36674.12  36685.39  36662.61   
9999  1645506180  2022-02-22 05:03:00  BTC/USD  36636.00  36636.00  36630.79   

         close  Volume BTC     Volume U

In [5]:
training_data = data_training.drop(['date', 'symbol','unix','Volume BTC'], axis = 1)
training_data.head()

,open,high,low,close,Volume USD
10001,36676.28,36676.28,36669.51,36671.91,6793.654320
10002,36646.81,36686.82,36646.81,36686.82,37505.202065
10003,36628.40,36642.71,36628.40,36642.70,894.448307
10004,36615.01,36649.31,36615.01,36630.20,8924.360315
10005,36653.02,36653.02,36611.81,36627.00,12436.162363


In [7]:
#MinMaxScaler is used to normalize the data
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
print(training_data)
#training_data


[[2.09808589e-01 2.03897890e-01 2.17589056e-01 2.10103020e-01
  4.86375413e-04]
 [2.09088160e-01 2.04157480e-01 2.17036287e-01 2.10467328e-01
  2.68509513e-03]
 [2.08638106e-01 2.03071093e-01 2.16587984e-01 2.09389310e-01
  6.40358847e-05]
 ...
 [2.30740140e-02 1.65475114e-02 3.12911000e-02 2.38253650e-02
  4.55360345e-03]
 [2.38592252e-02 1.66369148e-02 3.15136686e-02 2.35262957e-02
  6.29925432e-03]
 [2.23315836e-02 1.69787658e-02 3.10064362e-02 2.46888543e-02
  3.03203318e-02]]


In [8]:
X_train = [] 
Y_train = []
training_data.shape[0]

600781

In [ ]:
for i in range(10001, training_data.shape[0]):
    X_train=np.append(X_train,training_data[i-10001:i])
    Y_train=np.append(Y_train,training_data[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train.shape